In [196]:
import pandas as pd
import numpy as np
import csv
import seaborn as sns

In [167]:
BASE_PATH = '/Users/vrushankgunjur/documents/224n/final_proj'
GPT2_PATH = f'{BASE_PATH}/GPT2BaseResults.txt'
CUSTOMP_PATH = f'{BASE_PATH}/top_p.txt'
CUSTOMK_PATH = f'{BASE_PATH}/top_k.txt'
CUSTOMDOT_PATH = f'{BASE_PATH}/dotresults.txt'

In [175]:
# create df

def get_dataframe(path):
    res = []
    with open(path, 'r', encoding="utf-8") as results:
        lines = results.readlines()
        for line in lines[1:]:
            parts = line.split(',')
            #print(parts)

            full_prompt = ""
            #print(parts)
            prompt_pieces = parts[:-4]
            for piece in prompt_pieces:
                full_prompt += piece
            res.append({'Prompt': full_prompt, 'Max_Toxicity': float(parts[-4]), 'Num_Toxic': int(parts[-3]), 'Num_Analyzed': int(parts[-2]), 'Mean_Toxicity': float((parts[-1]))})
    #print(res)
    df = pd.DataFrame(res)
    df = df[:200]
    #df = df[:150]
    return df

In [176]:
def failure_percent(df):
    return 1 - (df['Num_Analyzed'].sum() / (len(df)*5))

In [190]:
def analyze_df(df):
    df['True_Mean'] = df['Mean_Toxicity'] * df['Num_Analyzed']
    df['True_Mean'] /= df['Num_Analyzed']
    mean_toxicity = df['True_Mean'].mean()
    
    print(f'N: {len(df)}')
    print(f'Mean Toxicity: {mean_toxicity}')

    mean_max_toxicity = df['Max_Toxicity'].mean()
    std_max_toxicity = df['Max_Toxicity'].std()
    print(f'Average Maximum Toxicity: {mean_max_toxicity}')
    print(f'Average Maximum STDEV: {std_max_toxicity}')

    # note that max toxicity is scuffed since it doesn't account for less than 5 sampled (jank responses)
    # this is particularly relevant to custom output


    #mean_num_toxic = df['Num_Toxic'].mean()
    #print(f'DOESNT WORK Mean # Toxic: {mean_num_toxic}')

    #print(len(df[df['Num_Toxic'] == 5]))

    percent_toxic1 = len(df[df['Num_Toxic'] != 0]) / (len(df))

    print(f'Percent Toxic (Toxic > 0 / Num Analyzed, their metric): {percent_toxic1}')
    #print(df['Num_Toxic'].sum())
    percent_toxic2 = df['Num_Toxic'].sum() / (df['Num_Analyzed'].sum())
    #print(df['Num_Toxic'])
    print(f'Percent Toxic (# Toxic tot / Num Analyzed, our metric): {percent_toxic2}')
    
    fp = failure_percent(df)
    print(f'Failure Percent: {fp}')
    
    return {'Mean Toxicity':mean_toxicity,'AvgMaxToxicity': mean_max_toxicity, 
            'AvgMaxToxicityStd': std_max_toxicity, 'PercentToxic1': percent_toxic1, 'PercentToxic2': percent_toxic2, 'FailurePercent': fp}
    #print(f'Failure Percent GPT2: {failure_percent(gpt2_df)}')
    #more_toxic = df[df['Num_Toxic'] > 1]
    #print(more_toxic)

In [203]:
customp_df = get_dataframe(CUSTOMP_PATH)
customk_df = get_dataframe(CUSTOMK_PATH)
customdot_df = get_dataframe(CUSTOMDOT_PATH)
gpt2_df = get_dataframe(GPT2_PATH)

data_list = []
print('GPT2: ')
data_list.append(analyze_df(gpt2_df))
print('----------')
print('CUSTOM TOP-P (DIST): ')
data_list.append(analyze_df(customp_df))
print('----------')
print('CUSTOM TOP-K (DIST): ')
#data_list.append(analyze_df(customk_df))
print('----------')
print('CUSTOM DOT (TOP-P): ')
data_list.append(analyze_df(customdot_df))


heatmap_df = pd.DataFrame(data_list)
heatmap_df
#sns.heatmap(heatmap_df['Mean Toxicity'])
#print(custom_df[custom_df['Num_Analyzed'] <= 2])

GPT2: 
N: 200
Mean Toxicity: 0.572677070274
Average Maximum Toxicity: 0.71276679473
Average Maximum STDEV: 0.1979964489595504
Percent Toxic (Toxic > 0 / Num Analyzed, their metric): 0.835
Percent Toxic (# Toxic tot / Num Analyzed, our metric): 0.639
Failure Percent: 0.0
----------
CUSTOM TOP-P (DIST): 
N: 200
Mean Toxicity: 0.539614384384
Average Maximum Toxicity: 0.66646441375
Average Maximum STDEV: 0.19719628144592458
Percent Toxic (Toxic > 0 / Num Analyzed, their metric): 0.755
Percent Toxic (# Toxic tot / Num Analyzed, our metric): 0.5877016129032258
Failure Percent: 0.008000000000000007
----------
CUSTOM TOP-K (DIST): 
----------
CUSTOM DOT (TOP-P): 
N: 200
Mean Toxicity: 0.5474182479009166
Average Maximum Toxicity: 0.6685957957700001
Average Maximum STDEV: 0.2094762551665043
Percent Toxic (Toxic > 0 / Num Analyzed, their metric): 0.765
Percent Toxic (# Toxic tot / Num Analyzed, our metric): 0.5945674044265593
Failure Percent: 0.006000000000000005


,Mean Toxicity,AvgMaxToxicity,AvgMaxToxicityStd,PercentToxic1,PercentToxic2,FailurePercent
0,0.572677,0.712767,0.197996,0.835,0.639000,0.000
1,0.539614,0.666464,0.197196,0.755,0.587702,0.008
2,0.547418,0.668596,0.209476,0.765,0.594567,0.006
